In [1]:
import pandas as pd
from ggplot import *
%matplotlib inline

# Load centroid and top 10 flows

In [11]:
dfCentroids = pd.read_csv('../01_OrigData/country_centroids_all.csv', sep = '\t')

In [12]:
dfCentroids.head()

,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,DSG,AFFIL,FIPS10,SHORT_NAME,FULL_NAME,MOD_DATE,ISO3136
0,33.000000,66.0,330000,660000,42STB1970055286,NI42-09,PCLI,NaN,AF,Afghanistan,Islamic Republic of Afghanistan,2009-04-10,AF
1,41.000000,20.0,410000,200000,34TDL1589839239,NK34-08,PCLI,NaN,AL,Albania,Republic of Albania,2007-02-28,AL
2,28.000000,3.0,280000,30000,31REL0000097202,NH31-15,PCLI,NaN,AG,Algeria,People's Democratic Republic of Algeria,2011-03-03,DZ
3,-14.333333,-170.0,-142000,-1700000,1802701,NaN,PCLD,US,AS,American Samoa,Territory of American Samoa,1998-10-06,AS
4,42.500000,1.5,423000,13000,31TCH7675006383,NK31-04,PCLI,NaN,AN,Andorra,Principality of Andorra,2007-02-28,AD


In [13]:
dfTop10Mig = pd.read_csv('../02_FlatData/MigrationPerCountry_Top10.csv', sep = ',')

In [14]:
dfTop10Mig.head()

,Country,Type,Coverage,OdName,Year,Migrants,Flow
0,Germany,Immigrants,Both,Turkey,1980,213126,Turkey - Germany
1,Germany,Immigrants,Both,Italy,1980,88865,Italy - Germany
2,Germany,Immigrants,Both,Poland,1980,67940,Poland - Germany
3,United States,Immigrants,Foreigners,Mexico,1980,56680,Mexico - United States
4,Australia,Immigrants,Both,United Kingdom,1980,43540,United Kingdom - Australia


# Let's merge them to get the Longitude and Latitude

In [42]:
dfTop10Mig.Type.unique()

array(['Immigrants'], dtype=object)

Thus, there is only the immigrant view included .. which means that the origin is always in the column "OdName" while the destination is in the column "Country"

It is a pity that we do not have country codes in here which means that we test if we can match based on the columns "Country" and "Short Name"

In [43]:
dfMerge1 = pd.merge(dfTop10Mig, dfCentroids[['SHORT_NAME', 'LAT', 'LONG']], how = 'left',
                    left_on = 'Country', right_on = 'SHORT_NAME')

In [44]:
dfMerge1['SHORT_NAME'].value_counts()

United States     126
Germany            88
Australia          34
Spain              22
United Kingdom     12
Italy              12
Canada              6
Croatia             1
Name: SHORT_NAME, dtype: int64

In [45]:
dfMerge1.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340 entries, 0 to 339
Data columns (total 10 columns):
Country       340 non-null object
Type          340 non-null object
Coverage      340 non-null object
OdName        340 non-null object
Year          340 non-null int64
Migrants      340 non-null int64
Flow          340 non-null object
SHORT_NAME    301 non-null object
LAT           301 non-null float64
LONG          301 non-null float64
dtypes: float64(2), int64(2), object(6)
memory usage: 29.2+ KB


Thus, there are still many countries where no match could have been found ...

In [46]:
dfMerge1[dfMerge1['SHORT_NAME'].isnull()]['Country'].value_counts()

Russian Federation    39
Name: Country, dtype: int64

Ok, with that we can deal. We only have to replace russia there ...

In [47]:
dfCentroids[dfCentroids['SHORT_NAME'].apply(lambda x: True if x.startswith('Ru') else False)]

,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,DSG,AFFIL,FIPS10,SHORT_NAME,FULL_NAME,MOD_DATE,ISO3136
188,60.0,100.0,600000,1000000,47VNG5577651833,NP47-14,PCLI,NaN,RS,Russia,Russian Federation,2007-02-28,RU


In [48]:
#Set it via hardcopy
dfMerge1.loc[dfMerge1['Country'] == 'Russian Federation', 'LAT'] = 60.0
dfMerge1.loc[dfMerge1['Country'] == 'Russian Federation', 'LONG'] = 100.0

In [50]:
dfMerge1[dfMerge1['LAT'].isnull()]

,Country,Type,Coverage,OdName,Year,Migrants,Flow,SHORT_NAME,LAT,LONG


In [51]:
#Delete the country column and rename the others
dfMerge1.drop('SHORT_NAME', inplace = True, axis = 1)
dfMerge1.rename(columns = {'LAT': 'dLat', 'LONG': 'dLong'}, inplace = True)

In [52]:
dfMerge1.head(1)

,Country,Type,Coverage,OdName,Year,Migrants,Flow,dLat,dLong
0,Germany,Immigrants,Both,Turkey,1980,213126,Turkey - Germany,51.5,10.5


In [62]:
dfMerge2 = pd.merge(dfMerge1, dfCentroids[['SHORT_NAME', 'LAT', 'LONG']], how = 'left',
                    left_on = 'OdName', right_on = 'SHORT_NAME')

In [63]:
dfMerge2.head()

,Country,Type,Coverage,OdName,Year,Migrants,Flow,dLat,dLong,SHORT_NAME,LAT,LONG
0,Germany,Immigrants,Both,Turkey,1980,213126,Turkey - Germany,51.500000,10.5000,Turkey,39.059012,34.911546
1,Germany,Immigrants,Both,Italy,1980,88865,Italy - Germany,51.500000,10.5000,Italy,42.833333,12.833333
2,Germany,Immigrants,Both,Poland,1980,67940,Poland - Germany,51.500000,10.5000,Poland,52.000000,20.000000
3,United States,Immigrants,Foreigners,Mexico,1980,56680,Mexico - United States,39.828175,-98.5795,Mexico,23.000000,-102.000000
4,Australia,Immigrants,Both,United Kingdom,1980,43540,United Kingdom - Australia,-25.000000,135.0000,United Kingdom,54.000000,-4.000000


Let's again check if we could map all countries ...

In [65]:
dfMerge2[dfMerge2['SHORT_NAME'].isnull()]['OdName'].value_counts()

Korea, Rep.           10
Russian Federation    10
Kyrgyz Republic        2
Lao PDR                1
Name: OdName, dtype: int64

Let's again update manually ...

In [76]:
#Let's start with Russia where we can just do the same as before
#Set it via hardcopy
dfMerge2.loc[dfMerge2['OdName'] == 'Russian Federation', 'LAT'] = 60.0
dfMerge2.loc[dfMerge2['OdName'] == 'Russian Federation', 'LONG'] = 100.0

In [77]:
dfMerge2[dfMerge2['LAT'].isnull()]['OdName'].value_counts()

Kyrgyz Republic    2
Lao PDR            1
Name: OdName, dtype: int64

Let's continue with Korea ...

In [78]:
dfCentroids[dfCentroids['SHORT_NAME'].apply(lambda x: True if x.startswith('South K') else False)]

,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,DSG,AFFIL,FIPS10,SHORT_NAME,FULL_NAME,MOD_DATE,ISO3136
212,37.0,127.5,370000,1273000,52SCF6653295924,NJ52-09,PCLI,NaN,KS,South Korea,Republic of Korea,2007-02-28,KR


In [79]:
#Set it via hardcopy
dfMerge2.loc[dfMerge2['OdName'] == 'Korea, Rep.', 'LAT'] = 37.0
dfMerge2.loc[dfMerge2['OdName'] == 'Korea, Rep.', 'LONG'] = 127.5

In [81]:
dfMerge2[dfMerge2['LAT'].isnull()]['OdName'].value_counts()

Kyrgyz Republic    2
Lao PDR            1
Name: OdName, dtype: int64

Now Kyrgyztan (?)

In [83]:
dfCentroids[dfCentroids['SHORT_NAME'].apply(lambda x: True if x.startswith('Kyr') else False)]

,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,DSG,AFFIL,FIPS10,SHORT_NAME,FULL_NAME,MOD_DATE,ISO3136
127,41.0,75.0,410000,750000,43TEF0000038757,NK43-08,PCLI,NaN,KG,Kyrgyzstan,Kyrgyz Republic,2007-02-28,KG


In [84]:
#Set it via hardcopy
dfMerge2.loc[dfMerge2['OdName'] == 'Kyrgyz Republic', 'LAT'] = 41.0
dfMerge2.loc[dfMerge2['OdName'] == 'Kyrgyz Republic', 'LONG'] = 75.0

In [85]:
dfMerge2[dfMerge2['LAT'].isnull()]['OdName'].value_counts()

Lao PDR    1
Name: OdName, dtype: int64

And we finish with Laos ...

In [86]:
dfCentroids[dfCentroids['SHORT_NAME'].apply(lambda x: True if x.startswith('Lao') else False)]

,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,DSG,AFFIL,FIPS10,SHORT_NAME,FULL_NAME,MOD_DATE,ISO3136
128,18.0,105.0,180000,1050000,48QWE0000090186,NE48-07,PCLI,NaN,LA,Laos,Lao People's Democratic Republic,2010-12-22,LA


In [87]:
#Set it via hardcopy
dfMerge2.loc[dfMerge2['OdName'] == 'Lao PDR', 'LAT'] = 18.0
dfMerge2.loc[dfMerge2['OdName'] == 'Lao PDR', 'LONG'] = 105.0

In [88]:
dfMerge2[dfMerge2['LAT'].isnull()]['OdName'].value_counts()

Series([], Name: OdName, dtype: int64)

Thus, everything is solved ..

In [89]:
#Delete the country column and rename the others
dfMerge2.drop('SHORT_NAME', inplace = True, axis = 1)
dfMerge2.rename(columns = {'LAT': 'oLat', 'LONG': 'oLong'}, inplace = True)

In [90]:
dfMerge2.head()

,Country,Type,Coverage,OdName,Year,Migrants,Flow,dLat,dLong,oLat,oLong
0,Germany,Immigrants,Both,Turkey,1980,213126,Turkey - Germany,51.500000,10.5000,39.059012,34.911546
1,Germany,Immigrants,Both,Italy,1980,88865,Italy - Germany,51.500000,10.5000,42.833333,12.833333
2,Germany,Immigrants,Both,Poland,1980,67940,Poland - Germany,51.500000,10.5000,52.000000,20.000000
3,United States,Immigrants,Foreigners,Mexico,1980,56680,Mexico - United States,39.828175,-98.5795,23.000000,-102.000000
4,Australia,Immigrants,Both,United Kingdom,1980,43540,United Kingdom - Australia,-25.000000,135.0000,54.000000,-4.000000


No let's write it out and have more fun in D3.js ...

In [91]:
dfMerge2.to_csv('../10_Code/MigrationPerCountry_Top10_v02.csv', sep = ',', index = False)

## Also adding an ID which gives the ordinal number of the flow in the respective year

In [95]:
lstDfs = []
for g in dfMerge2.groupby('Year'):
    g[1]['ID'] = range(1,11)
    lstDfs.append(g[1])
dfAddedId = pd.concat(lstDfs)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [97]:
dfAddedId.tail(11)

,Country,Type,Coverage,OdName,Year,Migrants,Flow,dLat,dLong,oLat,oLong,ID
329,Canada,Immigrants,Foreigners,China,2012,33024,China - Canada,60.000000,-96.000000,35.0,105.000000,10
330,United States,Immigrants,Foreigners,Mexico,2013,135028,Mexico - United States,39.828175,-98.579500,23.0,-102.000000,1
331,United States,Immigrants,Foreigners,China,2013,71798,China - United States,39.828175,-98.579500,35.0,105.000000,2
332,United States,Immigrants,Foreigners,India,2013,68458,India - United States,39.828175,-98.579500,20.0,77.000000,3
333,Italy,Immigrants,Both,Romania,2013,59347,Romania - Italy,42.833333,12.833333,46.0,25.000000,4
334,United States,Immigrants,Foreigners,Philippines,2013,54446,Philippines - United States,39.828175,-98.579500,13.0,122.000000,5
335,United Kingdom,Immigrants,Both,China,2013,50444,China - United Kingdom,54.000000,-4.000000,35.0,105.000000,6
336,United States,Immigrants,Foreigners,Dominican Republic,2013,41311,Dominican Republic - United States,39.828175,-98.579500,19.0,-70.666667,7
337,United Kingdom,Immigrants,Both,India,2013,35418,India - United Kingdom,54.000000,-4.000000,20.0,77.000000,8
338,United Kingdom,Immigrants,Both,Spain,2013,34489,Spain - United Kingdom,54.000000,-4.000000,40.0,-4.000000,9


In [98]:
dfAddedId.to_csv('../10_Code/MigrationPerCountry_Top10_v03.csv', sep = ',', index = False)

Somehow it did not work with the numbers, therefore let's use letters ...

In [100]:
lst = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

In [101]:
lstDfs = []
for g in dfMerge2.groupby('Year'):
    g[1]['ID'] = lst
    lstDfs.append(g[1])
dfAddedId2 = pd.concat(lstDfs)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [102]:
dfAddedId2.head()

,Country,Type,Coverage,OdName,Year,Migrants,Flow,dLat,dLong,oLat,oLong,ID
0,Germany,Immigrants,Both,Turkey,1980,213126,Turkey - Germany,51.500000,10.5000,39.059012,34.911546,a
1,Germany,Immigrants,Both,Italy,1980,88865,Italy - Germany,51.500000,10.5000,42.833333,12.833333,b
2,Germany,Immigrants,Both,Poland,1980,67940,Poland - Germany,51.500000,10.5000,52.000000,20.000000,c
3,United States,Immigrants,Foreigners,Mexico,1980,56680,Mexico - United States,39.828175,-98.5795,23.000000,-102.000000,d
4,Australia,Immigrants,Both,United Kingdom,1980,43540,United Kingdom - Australia,-25.000000,135.0000,54.000000,-4.000000,e


In [104]:
dfAddedId2.to_csv('../10_Code/MigrationPerCountry_Top10_v04.csv', sep = ',', index = False)